In [5]:
import math
from Corpus import *
import ipywidgets as widgets

In [6]:
# ==== INTERFACE ====

from IPython.display import display

subject_input = widgets.Text(
    value='Canada',
    description='Subject:'
)

articles_count = widgets.IntSlider(
    value=10,
    min=1,
    max=100,
    step=1,
    description='Nombre d\'articles :',
    style={'description_width': 'initial'}
)

button = widgets.Button(description="Lancer la recherche")

subject_value = subject_input.value
nb_articles_value = articles_count.value

def on_button_clicked(b):
    subject_value = subject_input.value
    nb_articles_value = articles_count.value
    Corpus.recherche(subject_value.lower(), math.floor(nb_articles_value / 2))

button.on_click(on_button_clicked)

display(subject_input, articles_count, button)

Text(value='Canada', description='Subject:')

IntSlider(value=10, description="Nombre d'articles :", min=1, style=SliderStyle(description_width='initial'))

Button(description='Lancer la recherche', style=ButtonStyle())

Titre : Influence of Magnetic Field on Effective Electron-Electron Interactions  in a Copper Wire
Auteur(s) : A. Anthore, F. Pierre, H. Pothier, D. Esteve, M. -H. Devoret
Date : 2001/09/17
URL : http://arxiv.org/abs/cond-mat/0109297v1

Titre : XMM -Newton observations of merging clusters of galaxies: A3921 and  A1750
Auteur(s) : E. Belsole, J-L. Sauvageot, R. Teyssier
Date : 2002/03/19
URL : http://arxiv.org/abs/astro-ph/0203297v1

Titre : Microwave probes Dipole Blockade and van der Waals Forces in a Cold  Rydberg Gas
Auteur(s) : R. Celistrino Teixeira, C. Hermann-Avigliano, Thanh Long Nguyen, T. Cantat-Moltrecht, Jean-Michel Raimond, S. Haroche, S. Gleyzes, M. Brune
Date : 2015/02/14
URL : http://arxiv.org/abs/1502.04179v1

Titre : Lorentz Dynamics on Closed 3-Manifolds
Auteur(s) : Charles Frances
Date : 2018/04/23
URL : http://arxiv.org/abs/1804.08695v1

Titre : An Optimised Flow for Futures: From Theory to Practice
Auteur(s) : Nicolas Chappe, Ludovic Henrio, Amaury Maillé, Matthieu